In [ ]:
# Downgrade sqlalchemy to a version less than 2.0
!pip install --upgrade 'sqlalchemy<2.0'

In [28]:
# imports
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
pd.options.display.max_rows = 10000

In [4]:
basics_url = "https://datasets.imdbws.com/title.basics.tsv.gz"
basics = pd.read_csv(basics_url, sep="\t", low_memory=False)

In [6]:
Akas_url = "https://datasets.imdbws.com/title.akas.tsv.gz"
Akas = pd.read_csv(Akas_url, sep="\t", low_memory=False)

In [7]:
Rating_url = "https://datasets.imdbws.com/title.ratings.tsv.gz"
Rating = pd.read_csv(Rating_url, sep="\t", low_memory=False)

In [8]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [9]:
Akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [10]:
Rating.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,2003
1,tt0000002,5.8,269
2,tt0000003,6.5,1896
3,tt0000004,5.5,178
4,tt0000005,6.2,2679


**Replacing \N by nan per data dictionary**

In [14]:
basics.replace({'\\N':np.nan},inplace=True)

In [15]:
Akas.replace({'\\N':np.nan},inplace=True)

In [16]:
Rating.replace({'\\N':np.nan},inplace=True)

**Cleaning Basics database**

In [17]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10250934 entries, 0 to 10250933
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 703.9+ MB


In [19]:
#Eliminate movies that are null for runtimeMinutes
basics['runtimeMinutes'].dropna(inplace=True)

In [21]:
basics['runtimeMinutes'].isna().sum()

7159618

In [24]:
basics['runtimeMinutes'].value_counts()

30                         256869
60                         193620
22                         168578
15                          85713
44                          77749
45                          74363
25                          70491
10                          66128
23                          65470
5                           63231
24                          62303
21                          60050
43                          56650
90                          54132
11                          50308
7                           49555
3                           48705
50                          47906
20                          47890
4                           47789
6                           47675
8                           45591
42                          45149
12                          43811
28                          40438
9                           38467
13                          35991
2                           31851
14                          31734
26            

In [25]:
#Eliminate movies that are null for genres
basics['genres'].isna().sum() 

458543

In [29]:
basics['genres'].value_counts()

Drama                               1173093
Comedy                               697026
Talk-Show                            638099
Drama,Romance                        491297
News                                 489719
Documentary                          489685
Reality-TV                           331687
Adult                                284902
News,Talk-Show                       224587
Short                                201506
Drama,Short                          193703
Family                               172517
Game-Show                            150827
Comedy,Short                         150412
Documentary,Short                    145538
Sport                                117229
Romance                              110451
Comedy,Talk-Show                     109147
Music                                106434
Comedy,Drama                          92879
Comedy,Drama,Romance                  91604
Animation                             73831
Action,Adventure,Animation      

In [30]:
basics['runtimeMinutes'].dropna(inplace=True)

In [ ]:
basics['titleType']==Movie